In [14]:
import json
from sklearn.model_selection import train_test_split

def split_data(jsonl_path, test_size=0.8, random_state=2024):
    # Read the jsonl file
    with open(jsonl_path, 'r') as file:
        data = [json.loads(line) for line in file]
    
    # Extract the images
    images = list(set([item['image'] for item in data]))
    
    # Split the data
    train_images, test_images = train_test_split(images, test_size=test_size, random_state=random_state)
    
    # Create training and test sets
    train_data = [item for item in data if item['image'] in train_images]
    test_data = [item for item in data if item['image'] in test_images]
    
    return train_data, test_data

# Example usage
jsonl_path = '/data/aofei/hallucination/CARES/HAM10000/HAM10000_factuality.jsonl'
ham_train_data, ham_test_data = split_data(jsonl_path)

In [15]:
len(ham_train_data), len(ham_test_data), ham_train_data[0]


(400,
 1600,
 {'question_id': 'ISIC_0034761',
  'options': 'A:scalp, B:ear, C:upper extremity, D:lower extremity',
  'fig_caption': 'C:upper extremity',
  'age': 45,
  'gender': 'female',
  'image': 'ham10000_testset/ISIC_0034761.jpg',
  'text': "This is a medical Question with several Options, and there is only one correct answer among these options. Please select the correct answer for the question. Remember, you can only select one option. The Question is:Which part of the body's skin is affected by pigmented lesion in this dermoscopy image?The candidate Options are:[A:scalp, B:ear, C:upper extremity, D:lower extremity]\n"})

In [16]:
#training2
new_ham_train_data = []
s = 0
for i in ham_train_data:
    template = dict()
    template['image'] = i['image']
    template['id'] = i['question_id'] + "_train" + str(s)
    template['conversations'] = []
    new_qa = {"from": "human", "value": "<image>\n" + i['text']}
    new_qa2 = {"from": "gpt", "value": str(i['fig_caption'])}
    template['conversations'] += [new_qa, new_qa2]
    new_ham_train_data.append(template)
    s+=1

new_ham_train_data[-1]

{'image': 'ham10000_testset/ISIC_0035481.jpg',
 'id': 'ISIC_0035481_train399',
 'conversations': [{'from': 'human',
   'value': '<image>\nThis is a medical Question with several Options, and there is only one correct answer among these options. Please select the correct answer for the question. Remember, you can only select one option. The Question is:What part of the body does the lesion in the image appear on?The candidate Options are:[A:trunk, B:face, C:ear, D:lower extremity]\n'},
  {'from': 'gpt', 'value': 'D:lower extremity'}]}

In [17]:
new_ham_test_data = []
s = 0
for i in ham_test_data:
    template = dict()
    template['conversations'] = []
    template['answer_type'] = "CLOSED"
    template['image'] = i['image']
    template['id'] = i['question_id'] + "_test" + str(s)
    new_qa = {"from": "human", "value": "<image>\n" + i['text']}
    new_qa2 = {"from": "gpt", "value": i['fig_caption']}
    template['conversations'] += [new_qa, new_qa2]
    new_ham_test_data.append(template)
    s += 1

new_ham_test_data[-1]

{'conversations': [{'from': 'human',
   'value': '<image>\nThis is a medical Question with several Options, and there is only one correct answer among these options. Please select the correct answer for the question. Remember, you can only select one option. The Question is:What type of abnormality is present in this image?The candidate Options are:[A:vascular lesions, B:basal cell carcinoma, C:melanocytic nevi, D:benign keratosis-like lesions]\n'},
  {'from': 'gpt', 'value': 'C:melanocytic nevi'}],
 'answer_type': 'CLOSED',
 'image': 'ham10000_testset/ISIC_0034672.jpg',
 'id': 'ISIC_0034672_test1599'}

In [18]:
import json
train_path = '/data/aofei/hallucination/CARES/HAM10000/training.json'
test_path = '/data/aofei/hallucination/CARES/HAM10000/test.json'
# Save training data
with open(train_path, 'w') as train_file:
    json.dump(new_ham_train_data, train_file, indent=4)

# Save test data
with open(test_path, 'w') as test_file:
    json.dump(new_ham_test_data, test_file, indent=4)

## IU-Xray

In [11]:
jsonl_path = '/data/aofei/hallucination/CARES/IU-Xray/iuxray_factuality.jsonl'
iu_train_data, iu_test_data = split_data(jsonl_path)

In [16]:
train_images = [i['image'] for i in iu_train_data]
test_images = [i['image'] for i in iu_test_data]
len(train_images), len(test_images)
common_images = set(train_images).intersection(set(test_images))
len(common_images), common_images

(0, set())

In [13]:
len(iu_train_data), len(iu_test_data), iu_train_data[0]

(1789,
 784,
 {'question': 'Does the cardiomediastinal silhouette appear normal in the chest X-ray? Please choose from the following two options: [yes, no]\n<image>',
  'answer': 'Yes.',
  'image': 'CXR3030_IM-1405/0.png',
  'question_id': 0,
  'text': 'Does the cardiomediastinal silhouette appear normal in the chest X-ray? Please choose from the following two options: [yes, no]\n<image>'})

In [17]:
# qs = qs.replace(DEFAULT_IMAGE_TOKEN, '').strip()

#training2
new_iu_train_data = []
s = 0
for i in iu_train_data:
    template = dict()
    template['image'] = i['image']
    template['id'] = str(i['question_id']) + "_train" + str(s)
    template['conversations'] = []
    new_qa = {"from": "human", "value": "<image>\n" + i['question'].replace("<image>", '').strip()}
    new_qa2 = {"from": "gpt", "value": str(i['answer'])}
    template['conversations'] += [new_qa, new_qa2]
    new_iu_train_data.append(template)
    s+=1

new_iu_train_data[-1]

{'image': 'CXR49_IM-2110/0.png',
 'id': '0_train1788',
 'conversations': [{'from': 'human',
   'value': '<image>\nAre the mediastinal contours within normal limits? Please choose from the following two options: [yes, no]'},
  {'from': 'gpt',
   'value': 'Yes, since the report does not mention any abnormalities of the mediastinum.'}]}

In [19]:
new_iu_test_data = []
s = 0
for i in iu_test_data:
    template = dict()
    template['conversations'] = []
    template['answer_type'] = "CLOSED"
    template['image'] = i['image']
    template['id'] = str(i['question_id']) + "_test" + str(s)
    new_qa = {"from": "human", "value": "<image>\n" + i['question'].replace("<image>", '').strip()}
    new_qa2 = {"from": "gpt", "value": i['answer']}
    template['conversations'] += [new_qa, new_qa2]
    new_iu_test_data.append(template)
    s += 1

new_iu_test_data[-1]

{'conversations': [{'from': 'human',
   'value': '<image>\nDoes the patient have a pleural effusion according to the X-ray? Please choose from the following two options: [yes, no]'},
  {'from': 'gpt', 'value': 'No.'}],
 'answer_type': 'CLOSED',
 'image': 'CXR1708_IM-0466/0.png',
 'id': '0_test783'}

In [22]:
import json
train_path = '/data/aofei/hallucination/CARES/IU-Xray/training.json'
test_path = '/data/aofei/hallucination/CARES/IU-Xray/test.json'
# Save training data
print(len(new_iu_train_data))
with open(train_path, 'w') as train_file:
    json.dump(new_iu_train_data, train_file, indent=4)

# Save test data
print(len(new_iu_test_data))
with open(test_path, 'w') as test_file:
    json.dump(new_iu_test_data, test_file, indent=4)

1789
784


## OminiVQA

In [15]:
import json
import os
data_path = "/data/aofei/hallucination/CARES/OmniMedVQA/omnimedvqa_factuality.jsonl"
with open(data_path, 'r') as file:
    data = [json.loads(line) for line in file]
data[0], len(data)

({'dataset': 'Adam Challenge',
  'question_id': 'Adam Challenge_0000',
  'question_type': 'Modality Recognition',
  'option_A': 'X-ray imaging',
  'option_B': 'Fundus photography',
  'option_C': 'Ultrasound imaging',
  'option_D': 'Magnetic resonance imaging (MRI)',
  'modality_type': 'Fundus Photography',
  'text': 'What imaging technique is employed to acquire this fundus image?',
  'question_idx': 1,
  'answer': 'Fundus photography',
  'image': 'Images/Adam Challenge/AMD/A0017.jpg'},
 12227)

In [17]:
data_w_iamge = []
for i in data:
    image_path = os.path.join("/data/aofei/hallucination/OmniMedVQA/VQA/raw/OmniMedVQA", i['image'])
    if os.path.exists(image_path):
        data_w_iamge.append(i)
len(data_w_iamge)

8797

In [18]:
def split_data(data, test_size=0.3, random_state=2024):
    # Extract the images
    images = list(set([item['image'] for item in data]))
    
    # Split the data
    train_images, test_images = train_test_split(images, test_size=test_size, random_state=random_state)
    
    # Create training and test sets
    train_data = [item for item in data if item['image'] in train_images]
    test_data = [item for item in data if item['image'] in test_images]
    
    return train_data, test_data


In [19]:
omni_train_data, omni_test_data = split_data(data_w_iamge)

In [20]:
len(omni_train_data), len(omni_test_data), omni_train_data[0]

(6155,
 2642,
 {'dataset': 'Adam Challenge',
  'question_id': 'Adam Challenge_0000',
  'question_type': 'Modality Recognition',
  'option_A': 'X-ray imaging',
  'option_B': 'Fundus photography',
  'option_C': 'Ultrasound imaging',
  'option_D': 'Magnetic resonance imaging (MRI)',
  'modality_type': 'Fundus Photography',
  'text': 'What imaging technique is employed to acquire this fundus image?',
  'question_idx': 1,
  'answer': 'Fundus photography',
  'image': 'Images/Adam Challenge/AMD/A0017.jpg'})

In [21]:
set([item['question_type'] for item in omni_train_data]), set([item['question_type'] for item in omni_test_data])

({'Anatomy Identification',
  'Disease Diagnosis',
  'Lesion Grading',
  'Modality Recognition'},
 {'Anatomy Identification',
  'Disease Diagnosis',
  'Lesion Grading',
  'Modality Recognition'})

In [22]:
#training2
new_omni_train_data = []
s = 0
for i in omni_train_data:
    template = dict()
    template['image'] = i['image']
    template['id'] = i['question_id'] + "_train" + str(s)
    template['conversations'] = []
    options = []
    for j in i.keys():
        if 'option' in j.lower():
            options.append(i[j])
    new_qa = {"from": "human", "value": "<image>\n" + i['text'] + " options: " + ", ".join(options)}
    new_qa2 = {"from": "gpt", "value": str(i['answer'])}
    template['conversations'] += [new_qa, new_qa2]
    new_omni_train_data.append(template)
    s+=1

new_omni_train_data[-1]

{'image': 'Images/MIAS/mdb248.png',
 'id': 'MIAS_0141_train6154',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat is the depicted abnormality in this image? options: Calcification, Hypertrophy, Ischemia, Hyperplasia'},
  {'from': 'gpt', 'value': 'Calcification'}]}

In [23]:
#training2
new_omni_test_data = []
s = 0
for i in omni_test_data:
    template = dict()
    template['image'] = i['image']
    template['id'] = i['question_id'] + "_train" + str(s)
    template['conversations'] = []
    options = []
    for j in i.keys():
        if 'option' in j.lower():
            template[j] = i[j]
            options.append(i[j])
    new_qa = {"from": "human", "value": "<image>\n" + i['text'] + " options: " + ", ".join(options)}
    new_qa2 = {"from": "gpt", "value": str(i['answer'])}
    template['conversations'] += [new_qa, new_qa2]
    new_omni_test_data.append(template)
    s+=1

new_omni_test_data[-1]

{'image': 'Images/MIAS/mdb214.png',
 'id': 'MIAS_0140_train2641',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat is the depicted abnormality in this image? options: Aneurysm, Hyperplasia, Calcification, Apoptosis'},
  {'from': 'gpt', 'value': 'Calcification'}],
 'option_A': 'Aneurysm',
 'option_B': 'Hyperplasia',
 'option_C': 'Calcification',
 'option_D': 'Apoptosis'}

In [24]:
import json
train_path = '/data/aofei/hallucination/CARES/OmniMedVQA/training.json'
test_path = '/data/aofei/hallucination/CARES/OmniMedVQA/test.json'
# Save training data
print(len(new_omni_train_data))
with open(train_path, 'w') as train_file:
    json.dump(new_omni_train_data, train_file, indent=2)

# Save test data
print(len(new_omni_test_data))
with open(test_path, 'w') as test_file:
    json.dump(new_omni_test_data, test_file, indent=2)

6155
2642


In [25]:
import os
s = 0
for i in new_omni_train_data:
    image_path = os.path.join('/data/aofei/hallucination/OmniMedVQA/VQA/raw/OmniMedVQA', i['image'])
    if os.path.exists(image_path):
        s += 1

In [26]:
s

6155

In [27]:
import os
s = 0
for i in new_omni_test_data:
    image_path = os.path.join('/data/aofei/hallucination/OmniMedVQA/VQA/raw/OmniMedVQA', i['image'])
    if os.path.exists(image_path):
        s += 1
s

2642

## OL3I

In [2]:
# !pip install h5py

In [13]:
import h5py
import os
from PIL import Image

# Open the file in read mode
with h5py.File('/data/aofei/hallucination/OL3I/l3_slices.h5', 'r') as file:
    # List all groups
    all_images_keys = list(file.keys())
    all_images = [file[key] for key in all_images_keys]

import matplotlib.pyplot as plt

# Open the file in read mode again to access the datasets
# with h5py.File('/data/aofei/hallucination/OL3I/l3_slices.h5', 'r') as file:
#     # Access and plot the first image as an example
#     first_image_key = all_images_keys[0]
#     first_image = file[first_image_key][:]
    
#     plt.imshow(first_image, cmap='gray')
#     plt.title(first_image_key)
#     plt.show()
#     # Create a directory to save the images if it doesn't exist
output_dir = '/data/aofei/hallucination/OL3I/images'
os.makedirs(output_dir, exist_ok=True)

# Open the file in read mode again to access the datasets
with h5py.File('/data/aofei/hallucination/OL3I/l3_slices.h5', 'r') as file:
    for key in all_images_keys:
        image_data = file[key][:]
        image = Image.fromarray(image_data)
        image = image.convert("L")  # Convert to grayscale
        image.save(os.path.join(output_dir, f"{key}.jpg"))


In [9]:
# len(list(file.keys()))
all_images_keys[:5]

['++NebvqGlMvCY5fNgoWqlPElDnew7MwkpQQgEATyhig=',
 '+0Hfk5_grXym69lMrp787ms41mZMAsOz+i9Dqt93baQ=',
 '+0OU1HKapBJZ2NTVTdN+F6jwSYooBJ0nDNG0TBA96hk=',
 '+0UEWiPmWjBh4_AYZvUyc5k8EZaJvKMrOCRXPmHoVFc=',
 '+0dgX7nnfhWN8PdzMFf2Msq3SR002Vapppq07JaJ+mU=']

In [11]:
all_images[0]


<Closed HDF5 dataset>

In [1]:
import json
import os
data_path = "/data/aofei/hallucination/CARES/OL3I/OL3I_factuality.jsonl"
with open(data_path, 'r') as file:
    data = [json.loads(line) for line in file]
data[0], len(data)

({'label': 0,
  'age': '68',
  'gender': 'male',
  'options': 'A:Yes, B:No',
  'answer': 'B:No',
  'question_id': 0,
  'text': 'Is ischemic heart disease detectable in this image? Please choose from the following two options: [yes, no]\n',
  'image': '5ov_JwsspJDgR9XHCiwCAqha6xvI+OhXz67m_tH4nyA=.jpg'},
 1000)

In [ ]:
data_w_iamge = []
for i in data:
    image_path = os.path.join('/data/aofei/hallucination/OL3I/images', i['image'])
    if os.path.exists(image_path):
        data_w_iamge.append(i)
len(data_w_iamge)

1000

In [3]:
import json
from sklearn.model_selection import train_test_split
def split_data(data, test_size=0.9, random_state=2024):
    # Extract the images
    images = list(set([item['image'] for item in data]))
    
    # Split the data
    train_images, test_images = train_test_split(images, test_size=test_size, random_state=random_state)
    
    # Create training and test sets
    train_data = [item for item in data if item['image'] in train_images]
    test_data = [item for item in data if item['image'] in test_images]
    
    return train_data, test_data

ol_train_data, ol_test_data = split_data(data_w_iamge)

In [4]:
len(ol_train_data), len(ol_test_data), ol_train_data[0]

(100,
 900,
 {'label': 0,
  'age': '40',
  'gender': 'male',
  'options': 'A:FRS and PCE, B:No specific types of abnormalities',
  'answer': 'B:No specific types of abnormalities',
  'question_id': 0,
  'text': 'What risk assessment methods can detect the specific type of pathological abnormalities shown in the images? Please choose from the following two options: [FRS and PCE, No specific types of abnormalities]\n',
  'image': 'qceAhHdhsVd4hnrgyjuMvEdppsy5U3XBXVDMM+qstzQ=.jpg'})

In [5]:
for i in ol_train_data:
    if "Please choose from" in i['text']:
        question = i['text'].split("Please choose from")[0].strip()
        if "yes" in i['options'].lower() and "no" in i['options'].lower():
            # if "yes" in i['answer'].lower():
            #     answer = "Yes"
            # else:
            #     answer = "No"
            answer = i['answer'].split(":")[-1].strip()
        else:
            question += " Options: " + i['options']
            answer = i['answer']
    else:
        question = i['text']
        if "yes" in i['options'].lower() and "no" in i['options'].lower():
            # if "yes" in i['answer'].lower():
            #     answer = "Yes"
            # else:
            #     answer = "No"
            answer = i['answer'].split(":")[-1].strip()
        else:
            question += " Options: " + i['options']
            answer = i['answer']
    i['text2'] = question
    i['answer2'] = answer
    
ol_train_data[0]

{'label': 0,
 'age': '40',
 'gender': 'male',
 'options': 'A:FRS and PCE, B:No specific types of abnormalities',
 'answer': 'B:No specific types of abnormalities',
 'question_id': 0,
 'text': 'What risk assessment methods can detect the specific type of pathological abnormalities shown in the images? Please choose from the following two options: [FRS and PCE, No specific types of abnormalities]\n',
 'image': 'qceAhHdhsVd4hnrgyjuMvEdppsy5U3XBXVDMM+qstzQ=.jpg',
 'text2': 'What risk assessment methods can detect the specific type of pathological abnormalities shown in the images? Options: A:FRS and PCE, B:No specific types of abnormalities',
 'answer2': 'B:No specific types of abnormalities'}

In [7]:
for i in ol_train_data[10:20]:
    print(i['text2'])

Does the image of the third lumbar vertebra show any signs of ischemic changes that would be consistent with Ischemic Heart Disease?
Does the axial image of the third lumbar vertebra present any visual indicators of heart-related conditions that could be indicative of Ischemic Heart Disease?
What does this axial image of the third lumbar vertebra contain that can help detect Ischemic Heart Disease? Options: A:Body composition imaging biomarkers for atherosclerotic cardiovascular disease, B:None
What does the axial image of the third lumbar vertebra indicate regarding the risk of Ischemic Heart Disease? Options: A:Increased risk, B:No indication of increased risk
What is the presence of any abnormal findings in the axial image of the third lumbar vertebra that could be related to Ischemic Heart Disease? Options: A:Abnormal findings are present, B:No abnormal findings related to Ischemic Heart Disease are present
Is there any correlation between the findings in this axial image of the th

In [8]:
#training2
new_ol_train_data = []
s = 0
for i in ol_train_data:
    template = dict()
    template['image'] = i['image']
    template['id'] = str(i['question_id']) + "_train" + str(s)
    template['conversations'] = []
    options = i['options']
    template['options'] = options
    new_qa = {"from": "human", "value": "<image>\n" + i['text2']}
    new_qa2 = {"from": "gpt", "value": str(i['answer2'])}
    template['conversations'] += [new_qa, new_qa2]
    new_ol_train_data.append(template)
    s+=1

new_ol_train_data[-1]

{'image': '7G28zEItafhQshce03rtbUoQihMrZB19M2wLSa+WORE=.jpg',
 'id': '0_train99',
 'conversations': [{'from': 'human',
   'value': '<image>\nAt 1 year follow-up, was the diagnosis of ischaemic heart disease negative for the individuals represented in the images?'},
  {'from': 'gpt', 'value': 'Yes'}],
 'options': 'A:No, B:Yes'}

In [9]:
new_ol_train_data[20]

{'image': 'EKyv7Swd6fmLvNJbJVUP4qIvp4qwPkrUT8oKiogdBJs=.jpg',
 'id': '0_train20',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat is the likelihood of detecting Ischemic Heart Disease from the image of the third lumbar vertebra? Options: A:High, B:Low to none'},
  {'from': 'gpt', 'value': 'B:Low to none'}],
 'options': 'A:High, B:Low to none'}

In [10]:
for i in ol_test_data:
    if "Please choose from" in i['text']:
        question = i['text'].split("Please choose from")[0].strip()
        if "yes" in i['options'].lower() and "no" in i['options'].lower():
            # if "yes" in i['answer'].lower():
            #     answer = "Yes"
            # else:
            #     answer = "No"
            answer = i['answer'].split(":")[-1].strip()
        else:
            question += " Options: " + i['options']
            answer = i['answer']
    else:
        question = i['text']
        if "yes" in i['options'].lower() and "no" in i['options'].lower():
            # if "yes" in i['answer'].lower():
            #     answer = "Yes"
            # else:
            #     answer = "No"
            answer = i['answer'].split(":")[-1].strip()
        else:
            question += " Options: " + i['options']
            answer = i['answer']
    i['text2'] = question
    i['answer2'] = answer
    
ol_test_data[0]

{'label': 0,
 'age': '68',
 'gender': 'male',
 'options': 'A:Yes, B:No',
 'answer': 'B:No',
 'question_id': 0,
 'text': 'Is ischemic heart disease detectable in this image? Please choose from the following two options: [yes, no]\n',
 'image': '5ov_JwsspJDgR9XHCiwCAqha6xvI+OhXz67m_tH4nyA=.jpg',
 'text2': 'Is ischemic heart disease detectable in this image?',
 'answer2': 'No'}

In [11]:
new_ol_test_data = []
s = 0
for i in ol_test_data:
    template = dict()
    template['image'] = i['image']
    template['id'] = str(i['question_id']) + "_test" + str(s)
    template['conversations'] = []
    Options_str = ["A", "B", "C", "D", "E"]
    for j in range(len(i['options'].split(","))):
        template["option_"+Options_str[j]] = i['options'].split(",")[j]
    new_qa = {"from": "human", "value": "<image>\n" + i['text2']}
    new_qa2 = {"from": "gpt", "value": str(i['answer2'])}
    template['conversations'] += [new_qa, new_qa2]
    new_ol_test_data.append(template)
    s+=1

new_ol_test_data[-1]

{'image': 'RyRzTqEcHscVjMw1ycPoMtF_PZEyCmr0TqK0xO1smn4=.jpg',
 'id': '0_test899',
 'conversations': [{'from': 'human',
   'value': '<image>\nDoes the axial image of the third lumbar vertebra show any signs of aortic atherosclerosis, which is associated with Ischemic Heart Disease?'},
  {'from': 'gpt',
   'value': 'No, there is no sign of aortic atherosclerosis in the image'}],
 'option_A': 'A:Yes',
 'option_B': ' there is visible aortic atherosclerosis',
 'option_C': ' B:No',
 'option_D': ' there is no sign of aortic atherosclerosis in the image'}

In [12]:
new_ol_test_data[-10]

{'image': 't7G2__0UdWdl+aKd9A3T+Tls4jvKs3BJ+DdtlnK7xH4=.jpg',
 'id': '0_test890',
 'conversations': [{'from': 'human',
   'value': '<image>\nDoes the axial image of the third lumbar vertebra present any visual indicators of heart-related conditions that could be indicative of Ischemic Heart Disease?'},
  {'from': 'gpt', 'value': 'No, there are no visual indicators'}],
 'option_A': 'A: Yes',
 'option_B': ' there are visual indicators',
 'option_C': ' B: No',
 'option_D': ' there are no visual indicators'}

In [13]:
import json
train_path = '/data/aofei/hallucination/CARES/OL3I/training.json'
test_path = '/data/aofei/hallucination/CARES/OL3I/test.json'
# Save training data
print(len(new_ol_train_data))
with open(train_path, 'w') as train_file:
    json.dump(new_ol_train_data, train_file, indent=2)

# Save test data
print(len(new_ol_test_data))
with open(test_path, 'w') as test_file:
    json.dump(new_ol_test_data, test_file, indent=2)

100
900
